In [20]:
import requests
import json
import pandas as pd

#### You need to research data from Itunes API of TOP paid applications in Russia for next tasks:
- define the model of data by spliting them to main data - some kind of daily rate of definite market research and guides which could present additional attributes of main data with relation's information for storing into DB
- make presentation of which tool you choose for scheduled ETL process into DB



### Loading data

In [21]:
url = 'https://rss.applemarketingtools.com/api/v2/ru/apps/top-paid/100/apps.json'

In [34]:
response = requests.get(url)
response.status_code

200

Носителем основных сущнестей является массив results
В него входят: 
    artistName - имя автора приложения
    id - айди, которое связывает автора, а также всю информацию о его приложении
    name - название приложения
    releaseDate - дата выхода приложения в релиз
    kind - в каком разделе находится эта программа
    artworkUrl100 - картинка приложения
    genres - массив с жанрами (стилями, категориями), к которым можно отнести приложение
        genreId - айди категории
        name - название
        url - ссылка на весь контент для этой категории
    url - ссылка на приложение

Если мы хотим положить это в нашу бд, то мы должно правильно разнести по сущностям эти атрибуты, произвести нормализацию

Предлагаю создать несколько таблиц:
    1) Таблица с авторами приложений
    2) Таблица с информацией о приложении
    3) Таблица с информацией о жанрах
    4) Таблица, которая будет связывать таблицу 2 и 3

В таблице 1 можно создать следующие атрибуты:
    id - уникальный айди автора
    name - имя автора
    name_l - имя автора на латиницу
    Можем хранить дополнительную информацию об авторе, например:
    website (желательно создать отдельный справочник для сайтов и хранить тут только ключ), 
    email (желательно создать отдельный справочник для стран и хранить тут только ключ), 
    country (желательно создать отдельный справочник для стран и хранить тут только ключ), created_at (дата создания автора), updated_at (дата последнего обновления автора)
    company - название студии, к которой принадлежит автор, тоже отдельный справочник тут не помешает, а тут будем хранить просто ключ

В таблице 2 необходимо хранить информацию об приложении
    app_id - уникальный айди приложения
    artist_id - ссылка на таблицу с авторами, тут мы храним ключ на поле id из первой таблицы
    app_name - название приложения
    app_name_l - название приложения на латинице
    release_date - дата выхода приложения (возможно, будет лучше тоже вынести в отдельный справочник, где будет храниться название, айди, а также даты с и по, так как приложение может быть на какое-то время удалено, а потом опять восстановленно, если нам это важно, то отдельный справочник тут будет нужен)
    url - ссылка на страницу в appstore этого приложения
    price - хотя этого атрибута не было в json, но как вариант это важная информация для нашей сущности
    size - тут может быть опять же ссылка на таблицу, где мы храним размеры приложений, так как они могут изменяться, или если мы настроим это иначе, то можем и тут хранить 
    downloads - кол-во скачиваний этого приложения
    currency - ссылка на таблицу с валютами, в которой продается это приложение
    pictures - ссылка на другую таблицу с картинками, или даже с json, смотря как мы храним картинки и где

Таблица 3 будет содержать разные жанры
    genre_id - айди жанра
    genre_name - название жанра
    genre_name_l - название жанра на латинице
    А также другая информация, которую мы можем хранить по жанрам, например дата создания жанра, 

Таблица 4 будет "посредником" между таблице приложений и жанрами. У нас будет связть м:м
    app_id - айди приложения
    genre_id - айди жанра


В качестве оркестратора можно использовать Apache Airflow. Со временм наша бд будет стремительно расти, а эта технология позволяет легко масштабировать процессы ETL, можем загружать данные из API, других бд, источников, чтобы наполнить наше хранилище. Также Airflow имеет достаточно хорошую систему мониторинга

